In [1]:
%run 10_ngram_common.ipynb

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
normal_speakers = ['MC01', 'MC02', 'MC03', 'MC04','FC01','FC02','FC03']
atypical_speakers = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']

# read torgo test speaker

In [3]:
# Read the dataset
data_df = pd.read_csv('torgo_new.csv')
dataset_csv = load_dataset('csv', data_files='torgo_new.csv')

speakers = data_df['speaker_id'].unique()

print(f'Speakers: {", ".join(speakers)}')

Speakers: F01, F03, F04, FC01, FC02, FC03, M01, M02, M03, M04, M05, MC01, MC02, MC03, MC04


In [32]:
import re

# Define a function to clean the text
def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    cleaned_text = re.sub(r'[^a-zA-Z0-9]', '', text.lower())
    return cleaned_text



In [36]:
import re

# Define a function to clean the text
def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
    return cleaned_text
    
# Apply the clean_text function to the 'text' column in data_df
# data_df['text'] = data_df['text'].apply(clean_text)

In [10]:
dataset_csv

DatasetDict({
    train: Dataset({
        features: ['session', 'audio', 'text', 'speaker_id', 'test_data'],
        num_rows: 16394
    })
})

In [42]:

speaker_texts = defaultdict(list)



# Initialize a defaultdict to store the texts for each speaker
speaker_texts_train = defaultdict(list)

# Iterate through each test speaker
for test_speaker in atypical_speakers:
    valid_speaker = 'F03' if test_speaker != 'F03' else 'F04'
    train_speaker = [s for s in speakers if s not in [test_speaker, valid_speaker]]
    torgo_dataset = DatasetDict()
    torgo_dataset['train'] = dataset_csv['train'].filter(lambda x: x in train_speaker, input_columns=['speaker_id'])
    torgo_dataset['validation'] = dataset_csv['train'].filter(lambda x: x == valid_speaker, input_columns=['speaker_id'])
    torgo_dataset['test'] = dataset_csv['train'].filter(lambda x: x == test_speaker, input_columns=['speaker_id'])

    torgo_dataset['train'] = torgo_dataset['train'].filter(lambda x: x['test_data'] == 0)
    torgo_dataset['validation'] = torgo_dataset['validation'].filter(lambda x: x['test_data'] == 0)
    torgo_dataset['test'] = torgo_dataset['test'].filter(lambda x: x['test_data'] == 1)

    # Filter the DataFrame to get the data for the train split for the selected train speakers
    test_data = torgo_dataset['test']
    
    # Append the texts from the filtered DataFrame to the corresponding speaker in the speaker_texts_train dictionary
    for text in test_data['text']:
        # print(text)
        new_text=clean_text(text)
        # print(new_text)
        speaker_texts[test_speaker].append(new_text)

# # Print the number of texts collected for each speaker
for speaker_id, texts in speaker_texts.items():
    print(f"{speaker_id}: {len(texts)}")
    for text in texts[:10]:
        print(text)
    print()

F01: 126
stick
except in the winter when the ooze or snow or ice prevents
up
know
he slowly takes a short walk in the open air each day
double
you wished to know all about my grandfather
knee
she had your dark suit in greasy wash water all year
car

F03: 592
beta
stubble
stubble
trace
goat
she had your dark suit in greasy wash water all year
well he is nearly ninetythree years old
knew
sip
alpha

F04: 367
fee
yet he still thinks as swiftly as ever
knew
trait
stubble
you wished to know all about my grandfather
chair
trace
go
hair

M01: 407
when he speaks his voice is just a bit cracked and quivers a trifle
trait
trouble
fee
raid
except in the winter when the ooze or snow or ice prevents
a long flowing beard clings to his chin
a long flowing beard clings to his chin
fair
bit

M02: 423
alpha
sip
dagger
fair
park
bubble
bit
well he is nearly ninetythree years old
trouble
chair

M03: 442
now
know
gadget
race
well he is nearly ninetythree years old
well he is nearly ninetythree years old
hai

In [47]:
from collections import defaultdict

# Initialize a defaultdict to store the texts for each speaker
speaker_texts_train = defaultdict(list)

# Iterate through each test speaker
for test_speaker in atypical_speakers:
    valid_speaker = 'F03' if test_speaker != 'F03' else 'F04'
    train_speaker = [s for s in speakers if s not in [test_speaker, valid_speaker]]
    torgo_dataset = DatasetDict()
    torgo_dataset['train'] = dataset_csv['train'].filter(lambda x: x in train_speaker, input_columns=['speaker_id'])
    torgo_dataset['validation'] = dataset_csv['train'].filter(lambda x: x == valid_speaker, input_columns=['speaker_id'])
    torgo_dataset['test'] = dataset_csv['train'].filter(lambda x: x == test_speaker, input_columns=['speaker_id'])

    torgo_dataset['train'] = torgo_dataset['train'].filter(lambda x: x['test_data'] == 0)
    torgo_dataset['validation'] = torgo_dataset['validation'].filter(lambda x: x['test_data'] == 0)
    torgo_dataset['test'] = torgo_dataset['test'].filter(lambda x: x['test_data'] == 1)

    # Filter the DataFrame to get the data for the train split for the selected train speakers
    train_data = torgo_dataset['train']
    
    # Append the texts from the filtered DataFrame to the corresponding speaker in the speaker_texts_train dictionary
    for text in train_data['text']:
        speaker_texts_train[test_speaker].append(clean_text(text))

# Print the number of texts collected for each speaker
for speaker_id, texts in speaker_texts_train.items():
    print(f"{speaker_id}: {len(texts)}")
    for text in texts[:2]:
        print(text)
    print()

F01: 9749
sheet
left

F03: 9551
pat
meat

F04: 9551
pat
meat

M01: 9519
pat
meat

M02: 9508
pat
meat

M03: 9493
pat
meat

M04: 9559
pat
meat

M05: 9594
pat
meat



In [48]:
speaker_texts_train

defaultdict(list,
            {'F01': ['sheet',
              'left',
              'air',
              'torn',
              'born',
              'sticks',
              'storm',
              'rage',
              'beat',
              'swarm',
              'usually minus several buttons',
              'store',
              'no',
              'grandfather likes to be modern in his language',
              'tip',
              'dug',
              'grow',
              'two',
              'two',
              'but he always answers banana oil',
              'air',
              'rake',
              'glow',
              'horn',
              'storm',
              'yes',
              'the quick brown fox jumps over the lazy dog',
              'feed',
              'swarm',
              'rain',
              'corn',
              'right',
              'feet',
              'ship',
              'form',
              'for',
              'train',
              'feed',
     

# calculate unique words

In [49]:
unique_words_per_speaker_train = {}

for speaker_id, texts in speaker_texts_train.items():
    unique_words = set()
    for text in texts:
        words = text.split()
        for word in words:
            cleaned_word = re.sub(r'[^a-zA-Z0-9]', '', word.lower())
            if cleaned_word:  
                unique_words.add(cleaned_word) 
    unique_words_per_speaker_train[speaker_id] = unique_words

for speaker_id, unique_words in unique_words_per_speaker_train.items():
    print(f"{speaker_id}: {len(unique_words)} unique words")

F01: 1272 unique words
F03: 1274 unique words
F04: 1274 unique words
M01: 1271 unique words
M02: 1269 unique words
M03: 1274 unique words
M04: 1274 unique words
M05: 1274 unique words


In [45]:
unique_words_per_speaker = {}

for speaker_id, texts in speaker_texts.items():
    unique_words = set()
    for text in texts:
        words = text.split()
        for word in words:
            cleaned_word = re.sub(r'[^a-zA-Z0-9]', '', word.lower())
            if cleaned_word:  
                unique_words.add(cleaned_word) 
    unique_words_per_speaker[speaker_id] = unique_words

for speaker_id, unique_words in unique_words_per_speaker.items():
    print(f"{speaker_id}: {len(unique_words)} unique words")

F01: 117 unique words
F03: 447 unique words
F04: 420 unique words
M01: 381 unique words
M02: 392 unique words
M03: 399 unique words
M04: 283 unique words
M05: 436 unique words


In [51]:
unique_words_per_speaker_train

{'F01': {'want',
  'acclaim',
  'rattle',
  'lawyers',
  'store',
  'future',
  'he',
  'few',
  'lid',
  'eat',
  'kittens',
  'bun',
  'sheep',
  'sake',
  'drugs',
  'bill',
  'dial',
  'mask',
  'behavior',
  'porch',
  'displayed',
  'corn',
  'whoop',
  'tip',
  'calls',
  'water',
  'hastily',
  'mom',
  'money',
  'damp',
  'gas',
  'chablis',
  'didnt',
  'places',
  'continental',
  'lamps',
  'drenched',
  'food',
  'medicine',
  'ladle',
  'boone',
  'sit',
  'laced',
  'pill',
  'look',
  'knife',
  'soul',
  'illegal',
  'last',
  'ray',
  'touchdown',
  'asked',
  'this',
  'hard',
  'burner',
  'sprain',
  'marriage',
  'screen',
  'petty',
  'ancient',
  'taking',
  'bash',
  'waves',
  'banana',
  'unit',
  'marshmallows',
  'railroads',
  'dusk',
  'trip',
  'shortage',
  'gold',
  'provides',
  'marvelously',
  'bureaucracy',
  'required',
  'later',
  'well',
  'history',
  'stuff',
  'company',
  'fill',
  'born',
  'come',
  'healthy',
  'aluminum',
  'shed',
  '

# vocabulary similarity

In [52]:
out_of_domain_words_per_speaker = {}

for speaker_id, unique_words in unique_words_per_speaker.items():
    overlap = len(unique_words & unique_words_per_speaker_train[speaker_id])
    similarity = overlap / len(unique_words) if len(unique_words) > 0 else 0
    out_of_domain_similarity = (1 - similarity)*100

    out_of_domain_words = {word for word in unique_words if word not in unique_words_per_speaker_train[speaker_id]}
    out_of_domain_words_per_speaker[speaker_id] = out_of_domain_words

    print(f"{speaker_id}: {out_of_domain_similarity:.2f}% dissimilarity with word_set_europarl")
    print(f"{speaker_id}: out_of_domain_words count: {len(out_of_domain_words)}")
    print(f"Out of domain words for {speaker_id}: {out_of_domain_words}")
    print()

F01: 54.70% dissimilarity with word_set_europarl
F01: out_of_domain_words count: 64
Out of domain words for F01: {'jacket', 'speaks', 'floor', 'winter', 'greasy', 'year', 'ice', 'trouble', 'jagged', 'suit', 'fee', 'twice', 'bat', 'skillfully', 'wash', 'respect', 'goat', 'utmost', 'double', 'flowing', 'ooze', 'raid', 'rave', 'except', 'trade', 'chin', 'zest', 'gadget', 'dagger', 'slowly', 'fair', 'clings', 'trifle', 'trace', 'race', 'alpha', 'thinks', 'quivers', 'swore', 'share', 'prevents', 'upon', 'bubble', 'feeling', 'trait', 'stubble', 'pronounced', 'park', 'weed', 'beta', 'bit', 'rate', 'takes', 'wished', 'cracked', 'stick', 'sip', 'car', 'beard', 'swiftly', 'organ', 'plays', 'bug', 'giving'}

F03: 62.64% dissimilarity with word_set_europarl
F03: out_of_domain_words count: 280
Out of domain words for F03: {'unpleasant', 'floor', 'jump', 'losing', 'goose', 'reek', 'something', 'year', 'court', 'light', 'echo', 'notch', 'juliet', 'bat', 'skillfully', 'overalls', 'goat', 'fleecy', 'do

# clean the text

# Out of domain analysis

In [ ]:
pattern = "keep_all"

In [67]:
atypical_speakers = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']

for speaker in atypical_speakers:
    arpa_file = f"torgo_lm_arpa_files/{speaker}_3gram_{pattern}.arpa"
    print(arpa_file)
    model=kenlm.Model(arpa_file) 
    

torgo_lm_arpa_files/F01_3gram_keep_all.arpa
torgo_lm_arpa_files/F03_3gram_keep_all.arpa
torgo_lm_arpa_files/F04_3gram_keep_all.arpa
torgo_lm_arpa_files/M01_3gram_keep_all.arpa
torgo_lm_arpa_files/M02_3gram_keep_all.arpa
torgo_lm_arpa_files/M03_3gram_keep_all.arpa
torgo_lm_arpa_files/M04_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa


In [69]:
import kenlm
# Initialize a dictionary to store perplexities per speaker
perplexities_per_speaker = {}

# Calculate perplexity for each speaker
for speaker_id, texts in cleaned_speaker_texts.items():
    arpa_file = f"torgo_lm_arpa_files/{speaker}_3gram_{pattern}.arpa"
    print(arpa_file)
    model=kenlm.Model(arpa_file) 
    
    perplexities = []
    for text in texts:
        perplexity = model.perplexity(text)
        perplexities.append(perplexity)
    avg_perplexity = np.mean(perplexities)
    perplexities_per_speaker[speaker_id] = avg_perplexity

# Print average perplexity for each speaker
for speaker_id, avg_perplexity in perplexities_per_speaker.items():
    print(f"Speaker {speaker_id}: Average Perplexity: {avg_perplexity}")

torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
Speaker F01: Average Perplexity: 17.028228872205332
Speaker F03: Average Perplexity: 24.043141069921028
Speaker F04: Average Perplexity: 18.58511757127206
Speaker M01: Average Perplexity: 18.142184382541494
Speaker M02: Average Perplexity: 18.794837074219913
Speaker M03: Average Perplexity: 18.568670409421134
Speaker M04: Average Perplexity: 17.49000922837935
Speaker M05: Average Perplexity: 21.27719734033508


Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/torgo_lm_arpa_files/M05_3gram_keep_all.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/torgo_lm_arpa_files/M05_3gram_keep_all.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/torgo_lm_arpa_files/M05_3gram_keep_all.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85-

In [71]:
mean_perplexity = sum(perplexities_per_speaker.values()) / len(perplexities_per_speaker)

print("Mean Perplexity:", mean_perplexity)

Mean Perplexity: 19.241173243536924


# output unigram

libri.apra
